In [35]:
import os
import csv
import json
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [36]:
def get_file_path(dirName):
    # # Get the list of all files in directory tree at given path

    # Get the list of all files in directory tree at given path
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]

    return listOfFiles


def make_df(list_of_files):
    # # Make a list of structures addreses
    # make a list of id and json of  tweets

    structures = []
    tweets = []
    for elem in list_of_files:
        elem = str(elem)
        if elem.endswith("structure.json"):
            structures += [elem]
        elif elem.endswith("json"):
            tweets += [elem]
    return structures, tweets


def make_cvs(direct):
    list_of_files = get_file_path(direct)
    structures, tweets = make_df(list_of_files)
    tweets_dic={}
    # print(list_of_files)
    with open('structure.csv', 'w', newline='') as t:
        for item in structures:
            f = open(item, "r")
            s = str(f.readline())
            t.write(s)
            f.close()
    t.close()
    with open('tweets.csv', 'w', newline='') as a:
        for item in tweets:
            # keep file az string
            # f = open(item, "r")
            # s = str(f.readline())
            # s = item[-23:-5] + ',' + s
            # a.write(s)
            # f.close()
            # just keep the main text
            with open(item) as f:
                data=json.load(f)
            tweets_dic[item[-23:-5]]=str(data["text"])
            s=item[-23:-5] + ','+str(data["text"])+'\n'
            a.write(s)
            f.close()

    a.close()
    '''write a dictionary and save it to pickle with id and text'''
    with open('tweets.pickle','wb') as handler:
        pickle.dump(tweets_dic,handler,protocol=pickle.HIGHEST_PROTOCOL)
    handler.close()
# def get_source(dr):
#     with open('source_tweets.csv','w',newline=' ')as a;
#         line=open(dr,"r")
#         s=str(line.readline())
'''combine id text and class for tweets
and give a array like :
ID|text|class

'''
def combine_text_id_classification(tweets,classifications,dev):
    '''

    :param tweets: id-text
    :param classifications: id-classification
    :return:
    '''
    id_text_class={}
    with open(classifications) as f:
        data = json.load(f)
        with open(tweets,'rb') as t:
            tweets_dic=pickle.load(t)
            data=data["subtaskaenglish"]
            for item in data:
                id_text_class[item]={"text":tweets_dic[item],"class":data[item]}
    type=dev+'_id_text_class.json'    
    with open(type, 'w') as fp:
        json.dump(id_text_class, fp)

def clean_text(test_string):
    '''

    :param test_string: get a string text
    :return: a list of tokens
    '''
    #remove usernames
    test_string = ' '.join(word for word in test_string.split(' ') if not word.startswith('@'))
    tokens=word_tokenize(test_string)
    # lower
    tokens = [w.lower() for w in tokens]
        #remove usernames
    tokens = words = [word for word in tokens if "@" not in word]
    # punctuation
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    #stop words
    stop_words = set(stopwords.words('english'))
    stop_words.add('https')
    words = [w for w in words if not w in stop_words]
    # stemmer
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    return words

In [37]:
def vectorize(path):
    '''
    vocabulary is 6000 words
    get all data vectorize data and return vector space
    :return:
    '''
    map_class={"comment":0,"deny":1,"support":2,"query":3}
    classification=[]
    with open(path) as f:
        tweets_json=json.load(f)
        corpus=[]
        for item in tweets_json:
            classification.append(map_class[tweets_json[item]["class"]])
            words=clean_text(tweets_json[item]["text"])
            s=""
            for token in words:
                s=s+" "+token
            corpus.append(s)
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return corpus,X,classification

In [38]:
classes='/Users/macbook/Desktop/reasearch/rumoureval2019/rumoureval-2019-training-data/tweets-train-key.json'
tweets='/Users/macbook/Desktop/reasearch/rumor/rumor/tweets.pickle'
combine_text_id_classification(tweets,classes,"train")
# vs,x,y=vectorize('/Users/macbook/Desktop/reasearch/rumoureval2019/rumoureval-2019-training-data/tweets-train-key.json')
# vs

In [41]:
vs,x,y=vectorize('/Users/macbook/Desktop/reasearch/rumor/rumor/train_id_text_class.json')

(4519, 5113)


In [42]:
x[0]

<1x5113 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [43]:
x.shape


(4519, 5113)

In [44]:
len(y)

4519

In [45]:
clf = MultinomialNB()
clf.fit(x,y)
clf.predict(x[3:5])

array([0, 0])

In [46]:
clf.predict(x[5:13])

array([0, 0, 0, 0, 0, 0, 0, 0])

In [47]:
clf.predict(x[5:133])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0,
       2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 1, 2, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0])

In [26]:
'''
fit data again
'''
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(vs)
clf = MultinomialNB()
clf.fit(x,y)

MultinomialNB()

In [50]:
'''
testing the dev data
'''
test_classes="/Users/macbook/Desktop/reasearch/rumoureval2019/rumoureval-2019-training-data/tweets-dev-key.json"
tweets='/Users/macbook/Desktop/reasearch/rumor/rumor/tweets.pickle'
combine_text_id_classification(tweets,test_classes,"test")

In [51]:
map_class={"comment":0,"deny":1,"support":2,"query":3}
classification=[]
with open('/Users/macbook/Desktop/reasearch/rumor/rumor/test_id_text_class.json') as f:
    tweets_json=json.load(f)
    corpus=[]
    for item in tweets_json:
        classification.append(map_class[tweets_json[item]["class"]])
        words=clean_text(tweets_json[item]["text"])
        s=""
        for token in words:
            s=s+" "+token
        corpus.append(s)
y_test=classification
x_test=vectorizer.transform(corpus)
x_test

<1049x5113 sparse matrix of type '<class 'numpy.int64'>'
	with 5949 stored elements in Compressed Sparse Row format>

In [52]:
y_test

[0,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 2,
 3,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 0,
 3,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 3,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 2,


In [53]:
predicted=clf.predict(x_test)
predicted

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
predicted[:100]

array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [55]:
import numpy as np
np.mean(predicted==y_test)

0.7102001906577693